<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/training_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Kernel setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%capture
!pip install wandb
!git clone https://github.com/Krankile/npmf.git

In [ ]:
%%capture
!cd npmf && git pull

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##General setup

In [ ]:
import os
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from operator import itemgetter

import numpy as np
import math
from numpy.ma.core import outerproduct
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm


import wandb as wb

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from sklearn.preprocessing import MinMaxScaler

from npmf.utils.colors import main, main2, main3
from npmf.utils.wandb import get_dataset, put_dataset
from npmf.utils.eikon import column_mapping

In [ ]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
np.random.seed(420)

# Create a Neural network class

In [ ]:
class MultivariateNetwork(nn.Module):
    def __init__(self, lag_len, cat_len, out_len, hidden_dim):
        super().__init__()

        self.pre = nn.Sequential(
            nn.Linear(lag_len, hidden_dim),
            nn.ReLU(),
        )

        self.predict = nn.Sequential(
            nn.Linear(hidden_dim + cat_len, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, out_len),
        )


    def forward(self, lags, cats):

        x = self.pre(lags)
        x = torch.cat((x, cats), dim=1)
        x = self.predict(x)

        return x

# Get some data

In [ ]:
stock_df = get_dataset("stock-oil-final:latest", project="master-test")
meta_df = get_dataset("meta-oil-final:latest", project="master-test")
fundamentals_df = get_dataset("fundamentals-oil-final:latest", project="master-test")
macro_df = get_dataset("macro-oil-final:latest", project="master-test")

wandb: Currently logged in as: kjartan (krankile). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact stock-oil-final:latest, 77.63MB. 1 files... Done. 0:0:0


# Define a dataset which can iterate through time 

We want to train our neural network like the person experiences the world. I.e. we have a window of time and look at recent financial rapports, macro variables to predict future market capitalization. We want to train multiple epochs over one time window and validation period, in this manner we will not have any "learned future" effect which could occur if one were to have epochs run over the all time windows. 

In [ ]:
class TimeDeltaDataset(Dataset):
    
    def _get_last_q_fundamentals(self, target, fundamental_df, q):
        for key, df in tqdm(fundamental_df.groupby(by="ticker")):
            
            padding = pd.DataFrame(np.empty((q, fundamental_df.loc[:,"revenue":].shape[1])),columns=df.loc[:,"revenue":].columns)
            padding[:] = np.nan
            padded_df = pd.concat([padding, df.loc[:,"revenue":]] , axis=0)
            
            target = pd.concat([target, padded_df.iloc[-q:,:]],axis=0)
        return target         
    
    def _get_global_local_column(self, stock_df):
        last_market_cap_col = pd.Series()
        for ticker, df in stock_df.groupby(by="ticker"):
            last_market_cap_col[ticker] = stock_df.market_cap.iloc[df.market_cap.last_valid_index()]

        min_max_scaler = MinMaxScaler()
        
        #Add column to learn relative values
        apple_market_cap = 2.687*(10**12) #ish as of may 2022 (USD)
        
        relative_to_global_market_column = last_market_cap_col / apple_market_cap
       
        relative_to_current_market_column = min_max_scaler.fit_transform(last_market_cap_col.to_numpy().reshape((-1,1)))
        relative_to_current_market_column = pd.Series(relative_to_current_market_column[:,0], index=last_market_cap_col.index) 

        return relative_to_global_market_column, relative_to_current_market_column, last_market_cap_col
    
    def _get_stocks_in_timeframe(self, stock_df, stock_dates, min_max_scaler=True):
        #TODO this needs a speedup when getting forecasts 
        to_be_transposed = pd.DataFrame(index=stock_dates)

        for i, (ticker, df) in enumerate(tqdm(stock_df.groupby(by="ticker"))):
            
            if min_max_scaler:
                min_max_scaler = MinMaxScaler()
                
                df.market_cap = min_max_scaler.fit_transform(df.market_cap.to_numpy().reshape((-1,1)))
            
            ticker_df = pd.DataFrame(df[["date", "market_cap"]]).set_index("date",drop=True).rename(columns={"market_cap":ticker})
            
            to_be_transposed = to_be_transposed.join(ticker_df)
            if i % 50 and i != 0:
                to_be_transposed = to_be_transposed.loc[~to_be_transposed.index.duplicated(),:]
        
        return to_be_transposed.T.loc[:,~to_be_transposed.T.columns.duplicated()]
    
    def _get_macro_normalized(self, legal_macro_df, macro_df):
        df = legal_macro_df.copy()
        for column in [c for c in legal_macro_df.columns if (c != "date") and ("_fx" not in c)]:
            df[column] = legal_macro_df[column]/ (int(math.ceil(macro_df[column].max() / 100.0)) * 100)
        return df

    def __init__(self, current_time, forecast_window, number_of_trading_days, q_quarterly_rapports, stock_df, fundamental_df, meta_df, macro_df):
        back_in_time = timedelta(number_of_trading_days)
        forecast_horizon = timedelta(forecast_window)

        legal_stock_df = stock_df[(stock_df.date >= current_time - back_in_time) & (stock_df.date < current_time)] #TODO change to current_time - stock__macro_days_lookback_days
        legal_fundamental_df = fundamental_df[fundamental_df.announce_date < current_time]
        legal_meta_df = meta_df.set_index("ticker")
        
        legal_macro_df = macro_df[(macro_df.date >= current_time - back_in_time) & (stock_df.date < current_time)] #TODO change to current_time - stock__macro_days_lookback_days
        
        #Important dimensions
        n_companies_with_fundamentals = len(legal_fundamental_df.ticker.unique())
        m_fundamentals = legal_fundamental_df.loc[:,"revenue":].shape[1]

        #Get last q fundamentals and return NA rows if they are still missing
        fundamental_df_all_quarters = pd.DataFrame(data=np.empty((0,m_fundamentals)),columns=fundamental_df.loc[:,"revenue":].columns)   
        fundamental_df_all_quarters = self._get_last_q_fundamentals(fundamental_df_all_quarters, legal_fundamental_df, q_quarterly_rapports)
        fundamentals = fundamental_df_all_quarters.to_numpy().reshape((n_companies_with_fundamentals, q_quarterly_rapports*m_fundamentals))
        
        #Construct columns for relative information 
        relative_to_global_market_column, relative_to_current_market_column, last_market_cap_col = self._get_global_local_column(stock_df)
        
        #Create dataframe
        historic_dates = pd.date_range(start=current_time-back_in_time, end=current_time, freq="D")
        forecast_dates = pd.date_range(start=current_time+timedelta(1), end=current_time+forecast_horizon, freq="D")

        fund_columns = []
        for i in range(q_quarterly_rapports):
            fund_columns.extend(fundamental_df.loc[0,"revenue":].index.to_series().map(lambda title: f"{title}_q=-{q_quarterly_rapports-i}"))    
        columns =  ["global_relative"] + ["peers_relative"] + fund_columns
        fundamental_df = pd.DataFrame(index=legal_fundamental_df.ticker.unique(), columns=columns)
        
        #Load data
        fundamental_df["peers_relative"]  = relative_to_current_market_column.loc[legal_fundamental_df.ticker.unique()]
        fundamental_df["global_relative"] = relative_to_global_market_column.loc[legal_fundamental_df.ticker.unique()]

        formated_stocks = self._get_stocks_in_timeframe(legal_stock_df, historic_dates)
         
        fundamental_df.loc[:,f"revenue_q={-q_quarterly_rapports}":"net_income_p_q=-1"] = fundamentals
        for q in range(q_quarterly_rapports,0,-1):
            fundamental_df.loc[:,f"revenue_q={-q}":f"fcf_q={-q}"] = fundamental_df.loc[:,f"revenue_q={-q}":f"fcf_q={-q}"].div(last_market_cap_col, axis=0)
            fundamental_df.loc[:,f"total_assets_q={-q}":f"total_current_liabilities_q={-q}"] = fundamental_df.loc[:,f"total_assets_q={-q}":f"total_current_liabilities_q={-q}"].div(fundamental_df.loc[:,f"total_assets_q={-q}"], axis=0)
            fundamental_df = fundamental_df.drop(columns=f"total_assets_q={-q}")

        self.stocks_and_fundamentals = formated_stocks.join(fundamental_df)
        
        # Get forecasts
        
        forecasts = stock_df[(stock_df.date > current_time) & (stock_df.date <= current_time + forecast_horizon)]
        
        forecasts_unormalized = self._get_stocks_in_timeframe(forecasts, forecast_dates, min_max_scaler=False)
        forecasts_normalized = forecasts_unormalized.div(last_market_cap_col, axis=0)
        self.forecast = forecasts_normalized.loc[self.stocks_and_fundamentals.index,:]
        
        #Join meta and stock-fundamentals
        legal_meta_df = legal_meta_df.loc[self.stocks_and_fundamentals.index,:] 
        legal_meta_df.loc[:,"exchange_code":"state_province_hq"] = legal_meta_df.loc[:,"exchange_code":"state_province_hq"].astype("category")
        legal_meta_df.loc[:,"economic_sector":"activity"] = legal_meta_df.loc[:,"economic_sector":"activity"].astype("category")
        
        legal_meta_df["founding_year"] = legal_meta_df["founding_year"]/2000  
        self.meta_df = legal_meta_df
        
        #Ready macro data
        self.macro_df = self._get_macro_normalized(legal_macro_df, macro_df).iloc[:,1:]


    def __len__(self):
        return self.stocks_and_fundamentals.shape[0]

    def __getitem__(self, idx):
        
        return (self.stocks_and_fundamentals.iloc[idx,:].values, self.legal_meta_df.iloc[idx,:].values, self.macro_df.T.values().reshape((1,-1))), self.forecast.iloc[idx,:]

In [ ]:
current_time = pd.to_datetime("2020-01-01")
number_of_quarterly_rapports = 4
number_of_trading_days = 365
forecast_window = 30

delta_set = TimeDeltaDataset(current_time, forecast_window, number_of_trading_days, number_of_quarterly_rapports, stock_df, fundamentals_df, meta_df, macro_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
100%|██████████| 862/862 [00:08<00:00, 101.56it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
100%|██████████| 864/864 [00:50<00:00, 17.20it/s]


# Run the loop! (Like Odd-Geir Lademo)

![picture](https://drive.google.com/uc?id=1Y55gFQSi4Baovmi0kUQGhbgGOBTI03E7)


In [ ]:
def train_multivar(model, optimizer, loss_fn, data_train, data_val, one_hot_encoding, batch_number, forecast_window, epochs, device):

# print(model)
train_losses = []
val_losses = []
it = tqdm(range(epochs), disable=True)
for epoch in it: 
    for run_type in ["train", "val"]:
        model.train(run_type == "train")

        if run_type == "train":
            optimizer.zero_grad()

            mu, sigma = 0, 0.1
            noise = np.random.normal(mu, sigma, data_train.shape)
            noise[:,-forecast_window:] = 0

            data_train = data_train + noise
            data_encoded = pd.concat([one_hot_encoding, data_train], axis=1, join="inner")

        else:
            data_encoded = pd.concat([one_hot_encoding, data_val], axis=1, join="inner")

        data_shuffled = torch.tensor(data_encoded.sample(frac=1).values, dtype=torch.float32)

        for batch in torch.split(data_shuffled, batch_number, dim=0):

            inputs = batch[:,:-forecast_window].to(device)
            actuals = batch[:,-forecast_window:].to(device)

            pred = model(inputs[:, 3:], inputs[:, 0:3])
            loss = loss_fn(pred, actuals)

            if run_type == "train":
                train_losses.append(loss.item())
                loss.backward()

                optimizer.step()
            else:
                val_losses.append(loss.item())

    it.set_postfix({"train_loss": np.mean(train_losses), "val_loss": np.mean(val_losses)})

return train_losses, val_losses

IndentationError: ignored

In [ ]:
def run_multivar(hidden_dim):
    #Training loop params
    forecast_window = 16
    amount_of_time_series = 999
    length_of_time_series = 500
    epochs = 200
    batch_size = 111
    


    #_, TS_signal, one_hot_encoding  = time_series_df(amount_of_time_series,length_of_time_series,periods, horisontal_shift, vertical_shift, forecast_window)

    df_train = TS_signal.iloc[-int(amount_of_time_series/3):,:]
    df_val = TS_signal.iloc[:-int(amount_of_time_series/3),:]

    loss_fn = nn.L1Loss()
    
    model = MultivariateNetwork(length_of_time_series, one_hot_encoding.shape[1], forecast_window, hidden_dim)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    return train_multivar(model, optimizer, loss_fn, df_train, df_val, one_hot_encoding, batch_size, forecast_window, epochs, device)